In [16]:
import json
import numpy as np
import os 
import pandas as pd

from tqdm import tqdm

OUT_DIR = "./out/type"
os.makedirs(OUT_DIR, exist_ok=True)


In [17]:
AIRPORT = "ksea"
BASE_DIR = '../../datasets/swim'

TRAJ_DIR = os.path.join(BASE_DIR, 'raw_trajectories', f'{AIRPORT}')

traj_files = [os.path.join(TRAJ_DIR, f) for f in os.listdir(TRAJ_DIR)]
print(traj_files[0])

data = pd.read_csv(traj_files[0])
data.head()

../../datasets/swim/raw_trajectories/ksea/KSEA_389_1673928000.csv


,Frame,ID,Altitude,Speed,Heading,Lat,Lon,Range,Bearing,Type,x,y
0,0,3838039,431.250,19.0,1.133789,47.445572,-122.306294,0.414762,-2.643944,0.0,-0.364454,-0.197991
1,0,3838156,431.250,12.6,198.216431,47.449816,-122.304810,0.137617,-0.675578,2.0,0.107389,-0.086059
2,0,3838073,431.250,17.3,146.458740,47.440690,-122.310184,1.031765,-2.645155,0.0,-0.907215,-0.491426
3,0,3837950,431.250,8.6,264.688110,47.463706,-122.307650,1.678735,-0.179768,0.0,1.651683,-0.300159
4,0,3838107,429.375,79.0,180.452637,47.443888,-122.307954,0.639378,-2.611640,0.0,-0.551675,-0.323201


In [18]:
agent_types = {}
total_count = 0
for f in tqdm(traj_files):
    data = pd.read_csv(f)

    unique_IDs = np.unique(data.ID)
    for ID in unique_IDs:
        atype = data[data.ID == ID].Type.astype(int)
        atype = atype[np.diff(atype, prepend=np.nan).astype(bool)].astype(str).tolist()
        key = ''.join(atype)

        if agent_types.get(key) is None:
            agent_types[key] = 0
            key_p = f"{key}_perc"
            agent_types[key_p] = 0.0
            
        agent_types[key] += 1
        total_count += 1
    

100%|██████████| 1398/1398 [06:38<00:00,  3.51it/s]


In [19]:

for k, v in agent_types.items():
    if "perc" in k: continue
    kp = f"{k}_perc"
    agent_types[kp] = round(v / total_count, 3)

agent_types['total'] = total_count

In [20]:
with open(f'{OUT_DIR}/{AIRPORT}.json', 'w') as f:
    json.dump(agent_types, f, indent=2)